In [51]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
appName("Sneha Spark Session").\
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [52]:
spark

In [53]:
order_schema = 'order_id long, order_date string, customer_id long, order_status string'

In [54]:
order_df = spark.read.\
format("csv").\
schema(order_schema).\
load("/public/trendytech/orders/orders_1gb.csv")

In [55]:
order_df.show()

+--------+--------------------+-----------+---------------+
|order_id|          order_date|customer_id|   order_status|
+--------+--------------------+-----------+---------------+
|       1|2013-07-25 00:00:...|      11599|         CLOSED|
|       2|2013-07-25 00:00:...|        256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|      12111|       COMPLETE|
|       4|2013-07-25 00:00:...|       8827|         CLOSED|
|       5|2013-07-25 00:00:...|      11318|       COMPLETE|
|       6|2013-07-25 00:00:...|       7130|       COMPLETE|
|       7|2013-07-25 00:00:...|       4530|       COMPLETE|
|       8|2013-07-25 00:00:...|       2911|     PROCESSING|
|       9|2013-07-25 00:00:...|       5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:...|       5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:...|        918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:...|       1837|         CLOSED|
|      13|2013-07-25 00:00:...|       9149|PENDING_PAYMENT|
|      14|2013-07-25 00:00:...|       98

In [56]:
spark.conf.get("spark.sql.files.maxPartitionBytes")

'134217728b'

In [57]:
(134217728 / (1024*1024))


128.0

In [58]:
spark.sparkContext.defaultParallelism

2

In [59]:
order_df.rdd.getNumPartitions()

9

In [60]:
new_order_df = order_df.repartition(1)

In [61]:
new_order_df.rdd.getNumPartitions()

1

In [62]:
new_order_df.write.format('csv'). \
mode('overwrite'). \
option("codec","org.apache.hadoop.io.compress.GzipCodec"). \
save("orders_Gz")

In [63]:
order_df_new = spark.read.format('csv').schema(order_schema).load("orders_Gz")

In [64]:
order_df_new.show()

+--------+--------------------+-----------+---------------+
|order_id|          order_date|customer_id|   order_status|
+--------+--------------------+-----------+---------------+
|   66159|2014-06-09 00:00:...|       3641|PENDING_PAYMENT|
|   66160|2014-06-09 00:00:...|       1507|       COMPLETE|
|   66161|2014-06-09 00:00:...|       2703|       COMPLETE|
|   66162|2014-06-10 00:00:...|      10268|PENDING_PAYMENT|
|   66163|2014-06-10 00:00:...|       6729|        PENDING|
|   66164|2014-06-10 00:00:...|       6943|       COMPLETE|
|   66165|2014-06-10 00:00:...|      12394|       COMPLETE|
|   66166|2014-06-10 00:00:...|        290|       COMPLETE|
|   66167|2014-06-10 00:00:...|       5828|PENDING_PAYMENT|
|   66168|2014-06-10 00:00:...|       7569|       COMPLETE|
|   66169|2014-06-10 00:00:...|       3927|        ON_HOLD|
|   66170|2014-06-10 00:00:...|      11218|         CLOSED|
|   66171|2014-06-10 00:00:...|       8803|PENDING_PAYMENT|
|   66172|2014-06-10 00:00:...|       34

In [65]:
order_df_new.rdd.getNumPartitions()

1

In [66]:
new_order_df.write.format('csv'). \
mode('overwrite'). \
option("codec","snappy"). \
save("orders_snappy")

In [67]:
order_df_new = spark.read.format('csv').schema(order_schema).load("orders_snappy")

In [68]:
order_df_new.rdd.getNumPartitions()

1

In [69]:
new_order_df.write. \
mode('overwrite'). \
save("orders_parquet_new")

In [70]:
order_df_new = spark.read.format('csv').load("orders_parquet_new")

In [71]:
order_df_new.rdd.getNumPartitions()

2

In [72]:
new_order_df.rdd.getNumPartitions()

1

In [73]:
final_order_df = new_order_df.repartition(20)

In [74]:
final_order_df.rdd.getNumPartitions()

20

In [75]:
final_order_df.write. \
format("csv"). \
mode('overwrite'). \
save("orders_final")

In [76]:
order_df= spark.read.format('csv').schema(order_schema).load("orders_final")

In [77]:
order_df.show()

+--------+--------------------+-----------+---------------+
|order_id|          order_date|customer_id|   order_status|
+--------+--------------------+-----------+---------------+
|   17523|2013-11-11 00:00:...|       9277|         CLOSED|
|   17523|2013-11-11 00:00:...|       9277|         CLOSED|
|   42679|2014-04-14 00:00:...|       1479|        ON_HOLD|
|   42679|2014-04-14 00:00:...|       1479|        ON_HOLD|
|   33615|2014-02-17 00:00:...|       8581|       COMPLETE|
|   33615|2014-02-17 00:00:...|       8581|       COMPLETE|
|   32649|2014-02-11 00:00:...|       7926|PENDING_PAYMENT|
|   32649|2014-02-11 00:00:...|       7926|PENDING_PAYMENT|
|   32649|2014-02-11 00:00:...|       7926|PENDING_PAYMENT|
|    3520|2013-08-14 00:00:...|      12180|       COMPLETE|
|    3520|2013-08-14 00:00:...|      12180|       COMPLETE|
|   39976|2014-03-29 00:00:...|       7426|         CLOSED|
|   39976|2014-03-29 00:00:...|       7426|         CLOSED|
|   27250|2014-01-10 00:00:...|      100

In [78]:
order_df.rdd.getNumPartitions()

10

In [79]:
spark.conf.get("spark.sql.files.openCostInBytes")

'4194304'

In [80]:
(4194304 / (1024*1024))

4.0

In [81]:
final_order_df = new_order_df.repartition(500)

In [82]:
final_order_df.rdd.getNumPartitions()

500

In [83]:
final_order_df.write. \
format("csv"). \
mode('overwrite'). \
save("orders_final_new")

In [84]:
order_df= spark.read.format('csv').schema(order_schema).load("orders_final_new")

In [85]:
order_df.rdd.getNumPartitions()

24